In [1]:
import numpy as np
import pandas as pd

from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.database.queries import *
from src.instance.InstanceList import InstanceList
from src.instance.TSP_Instance import TSP_from_index_file
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver

In [2]:
generators = [
    "cluster_netgen",
    "compression",
    "expansion",
    "explosion",
    "grid",
    "cluster",
    "implosion",
    "linearprojection",
    "rotation",
    "uniform_portgen",
]

test_instances = TSP_from_index_file(filepath=DATA_DIR / "TSP" / "TEST" / "index.json")

id_to_generator_name = {
    instance.id(): instance.filepath.parts[-2] for instance in test_instances
}

id_to_name = {
    instance.id(): instance.filepath.parts[-1] for instance in test_instances
}

In [3]:
PREFIX = "run-plain-30"
# PREFIX = "run-30-sur-50"

In [4]:
rows = []
for idx, db_path in enumerate(DATABASE_DIR.glob(f"{PREFIX}-*.db")):
    db = DB(db_path)
    results = pd.read_sql_query("SELECT * FROM results", db._conn)
    results_config = results.loc[results["prefix"].str.startswith("config"), :]
    config_cutoff_ratio = (results_config["cost"] == TSP_LKH_Solver.MAX_COST).mean()
    config_error_ratio = (results_config["error"] == 1).mean()
    config_surrogate_ratio = (results_config["surrogate"] == 1).mean()

    results_test = results.loc[results["prefix"].str.startswith("test"), :]
    test_cutoff_ratio = (results_test["cost"] == TSP_LKH_Solver.MAX_COST).mean()
    test_error_ratio = (results_test["error"] == 1).mean()

    rows.append({
        "name": db_path.name,
        "config_cutoff_ratio": config_cutoff_ratio,
        "config_error_ratio": config_error_ratio,
        "config_surrogate_ratio": config_surrogate_ratio,
        "test_cutoff_ratio": test_cutoff_ratio,
        "test_error_ratio": test_error_ratio,
    })

pd.DataFrame(rows).style.format(precision=2)

,name,config_cutoff_ratio,config_error_ratio,config_surrogate_ratio,test_cutoff_ratio,test_error_ratio
0,run-plain-30-951417.db,0.51,0.08,0.00,0.02,0.00
1,run-plain-30-951418.db,0.51,0.08,0.00,0.08,0.00
2,run-plain-30-951419.db,0.51,0.07,0.00,0.30,0.09
3,run-plain-30-951420.db,0.52,0.07,0.00,0.34,0.00
4,run-plain-30-951421.db,0.50,0.08,0.00,0.03,0.00
5,run-plain-30-951422.db,0.50,0.09,0.00,0.03,0.00
6,run-plain-30-951423.db,0.49,0.10,0.00,0.21,0.04
7,run-plain-30-951424.db,0.53,0.08,0.00,0.00,0.00
8,run-plain-30-951425.db,0.48,0.09,0.00,0.01,0.00
9,run-plain-30-951426.db,0.51,0.10,0.00,0.04,0.00


In [18]:
frames = []

def agg_cost(x):
    count = x.shape[0]
    x = x[x < TSP_LKH_Solver.MAX_COST]
    idx = count // 2
    if x.shape[0] <= idx:
        return x.iloc[-1]
    return x.iloc[idx]
    

for idx, db_path in enumerate(DATABASE_DIR.glob(f"{PREFIX}-*.db")):
    db = DB(db_path)
    results = pd.read_sql_query("SELECT * FROM results", db._conn)
    series = (
        results.loc[results["prefix"].str.startswith("test")]
        .groupby(["instance_id", "prefix"])["cost"]
        .min()
        .reset_index()
        .groupby("instance_id")["cost"]
        .agg(agg_cost)
    )
    frames.append(series)
    

df = (
    pd.concat(frames, axis=1)
    .mean(axis=1)
    .round(2)
    .rename(PREFIX)
    .to_frame()
)

df["generator"] = df.index.map(id_to_generator_name)
df["name"] = df.index.map(id_to_name)
df = df.pivot_table(index="generator", columns="name", values=PREFIX).loc[generators, :]
df["mean"] = df.mean(axis=1)

In [19]:
df.to_excel("tmp1.xlsx")

In [6]:
# 
df

name,000.tsp,001.tsp,002.tsp,003.tsp,004.tsp,mean
generator,,,,,,
cluster_netgen,0.11,0.41,0.48,0.38,0.81,0.438
compression,0.26,0.21,1.53,0.23,0.16,0.478
expansion,0.31,0.14,0.77,0.54,0.39,0.430
explosion,0.87,0.12,0.15,0.19,0.09,0.284
grid,0.73,0.15,0.46,0.56,0.12,0.404
cluster,0.56,0.11,0.13,0.15,0.15,0.220
implosion,0.09,0.20,1.13,0.53,0.13,0.416
linearprojection,0.50,0.44,0.20,0.10,0.16,0.280
rotation,0.31,1.22,0.14,0.13,0.13,0.386


In [53]:
df.mean().round(2)

run-30-sur-50    0.52
dtype: float64

In [54]:
times = {}

for idx, db_path in enumerate(DATABASE_DIR.glob(f"{PREFIX}-*.db")):
    db = DB(db_path)
    results = pd.read_sql_query("SELECT * FROM results", db._conn)
    times[idx] = results.loc[
        results["prefix"].str.startswith("config")
        & results["cached"].eq(0)
        & results["surrogate"].eq(0),
        "time",
    ].sum()
    
np.round(pd.Series(times).mean(), 2)

20640.65

In [55]:
pd.Series(times).mean() / 3600

5.73351327659766